<a href="https://colab.research.google.com/github/KULL-Centre/ColabCALVADOS/blob/main/CALVADOS_simulate_and_reweight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CALVADOS Colab: Reweighting ensembles using SAXS data
This Colab notebook enables running molecular dynamics (MD) simulations of intrinsically disordered proteins (IDPs) and multi-domain proteins (MDPs) and to study their conformational ensembles [1,2,3].

MD simulations employ the coarse-grained force fields CALVADOS 2 or 3 [1,2], where C$_\alpha$ and center-of-mass (COM) representation will be used for IDPs and MDPs, respectively.

Simulations are run using OpenMM [4] based on user-defined `temperature`, `ionic strength`, `pH`, `sequence`, and `domain boundary` (only for MDPs). Detailed instructions can be found in Cell 2.

Coarse-grained trajectories are converted to all-atom trajectories using cg2all [5].

Small angle X-ray scattering curves are calculated using _polynomial expansions of protein structures and interactions_ (Pepsi)-SAXS [6].

Bayesian/Maximum Entropy Reweighting is performed using the BME Python library [7,8] and estimating the experimental error of SAXS intensities using the Bayesian indirect Fourier transformation [9].

### Usage
We recommend running MD simulations run on a single GPU. To enable GPU select `Runtime` from the menu, then `Change runtime type` and select `GPU`.

Note: Cells for preliminary operations should be executed one by one to prevent crashes. This notebook uses condacolab, whose installation will cause a kernel restart. Because of this, a crash warning will happen during preliminary operations if you execute all cells at once.

### References
If you use this notebook, you may consider citing [1] for simulations using CALVADOS 2, [2] for simulations using CALVADOS 3, [3] for the machine learning predictor of scaling exponents, [4] for OpenMM, [5] for cg2all, [6] for Pepsi-SAXS, and [7,8,9] for BME reweighting based on quantitative comparisons between experimental and predicted SAXS curves.  

1. G. Tesei and K. Lindorff-Larsen __Improved predictions of phase behaviour of intrinsically disordered proteins by tuning the interaction range [version 2; peer review: 2 approved]__ _Open Res Eur._ 2023 2(94) DOI: https://doi.org/10.12688/openreseurope.14967.2

2. F. Cao, S. von Bülow, G. Tesei, and K. Lindorff‐Larsen (2024). __A coarse‐grained model for disordered and multi‐domain proteins.__ _Protein Sci._ 33(11):e5172 DOI: https://doi.org/10.1002/pro.5172

3. G. Tesei, A. I. Trolle, N. Jonsson, J. Betz, F. E. Knudsen, F. Pesce, K. E. Johansson, K. Lindorff-Larsen __Conformational ensembles of the human intrinsically disordered proteome__ _Nature._ 2024 626:897–904 2023.05.08.539815 DOI: https://doi.org/10.1038/s41586-023-07004-5

4. P. Eastman, J. Swails, J. D. Chodera et al. __OpenMM 7: Rapid development of high performance algorithms for molecular dynamics__ _PLoS Comput Biol._ 2017 13(7):e1005659 DOI: https://doi.org/10.1371/journal.pcbi.1005659

5. L. Heo and M. Feig (2023). __One particle per residue is sufficient to describe all-atom protein structures.__ _bioRxiv_, 2023-05.

6. S. Grudinin, M. Garkavenko, and A. Kazennov __Pepsi-SAXS: an adaptive method for rapid and accurate computation of small-angle x-ray scattering profiles__ _Acta Crystallogr D Struct Biol._ 2017 73:449–464 DOI: https://doi.org/10.1107/S2059798317005745

7. S. Bottaro, T. Bengtsen, and K. Lindorff-Larsen __Integrating Molecular Simulation and Experimental Data: A Bayesian/Maximum Entropy Reweighting Approach__ _Methods Mol Biol._ 2020 2112:219–240 DOI: https://doi.org/10.1007/978-1-0716-0270-6_15

8. F. Pesce and K. Lindorff-Larsen __Refining conformational ensembles of flexible proteins against small-angle x-ray scattering data__ _Biophys J._ 2021 120(22):5124–5135 DOI: https://doi.org/10.1016/j.bpj.2021.10.003

9. A. H. Larsen and M. C. Pedersen __Experimental noise in small-angle scattering can be assessed using the Bayesian indirect Fourier transformation__ _J Appl Cryst._ 2021 54:1281–1289 DOI: https://doi.org/10.1107/S1600576721006877

In [ ]:
# @title 1. Set the environment for simulation

# @markdown Attention! A prompt of "Your session crashed for an unknown reason" will appear after running this block. This is required for all packages to work properly. Please ignore it and move on.

# @markdown If you have many proteins to run, change the inputs in cell 2, 3 and 4 for each protein as you need. You only need to run cell 1 once.
import subprocess
import locale
locale.getpreferredencoding = lambda: "UTF-8"
subprocess.run('pip install py3Dmol'.split())
subprocess.run('pip install MDAnalysis'.split())
subprocess.run('pip install mdtraj'.split())
subprocess.run('pip install -q condacolab'.split())
import condacolab
condacolab.install()
subprocess.run('mamba install openmm -c conda-forge --yes'.split())
subprocess.run('pip install wget'.split())
subprocess.run('pip install fastprogress'.split())
subprocess.run('mamba install pandas'.split())
subprocess.run('mamba install pydantic'.split())
# !pip install "dgl<=1.0"  # cpu for cg2all

In [ ]:
# @title 2. Configure your protein

#@markdown In data/SAXS/exp_conditions.csv at github.com/KULL-Centre/ColabCALVADOS you can find __temperature__, __ionic strength__, __pH__, __domain boundaries__, and __sequence__ information for SAXS measurements of IDPs and 13 MDPs (see DOI: 10.1016/j.bpj.2022.12.013, 10.1002/pro.5172). In the same folder, you can also find structure files (*.pdb) for 13 MDP.

#@markdown Choose the protein you want to work with and download its “.fasta” file, ".pdb" files (for MDPs) and SAXS data (“.dat” file). You can also use your own data for the protein of your choice.

import numpy as np
import os
import shutil
import ipywidgets as widgets
import warnings
import wget
import yaml
from google.colab import files

warnings.filterwarnings('ignore')

example = True  #@param {type:"boolean"}
# @markdown If `example` is checked, all the inputs below in this cell will be ignored. In this case, [THB_C2](https://www.biorxiv.org/content/10.1101/2024.02.03.578735v3) (MDP) will serve as an example for tutorial with pre-defined settings.

isIDP = False #@param {type:"boolean"}
name = "hSUMO_hnRNPA1S" #@param {type:"string"}
sequence = "MGSSHHHHHHGSGLVPRGSASMSDSEVNQEAKPEVKPEVKPETHINLKVSDGSSEIFFKIKKTTPLRRLMEAFAKRQGKEMDSLRFLYDGIRIQADQTPEDLDMEDNDIIEAHREQIGGMSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNARPHKVDGRVVEPKRAVSREDSQRPGAHLTVKKIFVGGIKEDTEEHHLRDYFEQYGKIEVIEIMTDRGSGKKRGFAFVTFDDHDSVDKIVIQKYHTVNGHNCEVRKALSKQEMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGNYNNQSSNFGPMKGGNFGGRSSGPYGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF" #@param {type:"string"}
L = int(np.ceil((len(sequence) - 1) * 0.38 + 4))
if L > 900:  # nm
    raise Exception("Too large box! Floats might overflow in PDB file. You can try to modify the 'L' parameter in this block yourself.")
temperature = 293.15 #@param {type:"number"}
ionic_strength = 0.1 #@param {type:"number"}
pH = 7.5 #@param {type:"number"}
cutoff = 2.0  # nm
#@markdown <i>*Units: temperature [K], ionic strength [M]<i>

#@markdown <b>Define domain boundary -- ignore this if `isIDP` is toggled on:

#@markdown Residue ranges are delimited by `-`, e.g. residues 1 to 200 are `1-200`;

#@markdown Separate domains are delimited by `,`, e.g. `1-200,201-400`;

#@markdown Discontinuous segments are delimited by `_`, e.g. `1-200,201-300_350-400`.
domain_boundary = "44-114,132-209,224-298" #@param {type:"string"}
input_pae = False
k_restraint = 700  # unit:KJ/(mol*nm^2)
eps_factor = 0.2
CALVADOS_version = 3
discard_first_nframes = 10

if example:
    name = "THB_C2"
    isIDP = False
    sequence = "GPGSEDVWEILRQAPPSEYERIAFQYGVTDLRGMLKRLKGMRRDEKKSTAFQKKLEPAYQVSKGHKIRLTVELADHDAEVKWLKNGQEIQMSGSKYIFESIGAKRTLTISQCSLADDAAYQCVVGGEKCSTELFVKE"
    L = int(np.ceil((len(sequence) - 1) * 0.38 + 4))
    temperature = 295.15
    ionic_strength = 0.15
    pH = 6.5
    domain_boundary = "6-42,50-137"

use_pdb = False if isIDP else True
use_hnetwork = False if isIDP else True
use_ssdomains = False if isIDP else True

if not isIDP:
    domain_resNum = []  # start from 1
    for domain in domain_boundary.split(","):
        domain = domain.split("_")
        if len(domain) == 1:  # no break point
            domain_resNum.append(list(range(int(domain[0].split('-')[0]), int(domain[0].split('-')[1]) + 1)))
        else:
            tmp_list = []
            for subdomain in domain:
                tmp_list += list(range(int(subdomain.split('-')[0]), int(subdomain.split('-')[1]) + 1))
            domain_resNum.append(tmp_list)
else:
    domain_boundary = None
    domain_resNum = None
    path2pdb = ''

print(f"{'IDP' if isIDP else 'MDP'} name: {name}")
print(f"sequence: {sequence}")
print(f"simulation box: [{L}, {L}, {L}] nm")
print(f"temperature: {temperature} K")
print(f"ionic strength: {ionic_strength} M")
print(f"pH: {pH}")
if not isIDP:
    print(f"There are {len(domain_resNum)} restrained domains (1-based): ")
    for d in domain_resNum:
        print(d)
if not os.path.isdir(f"{name}"):
    os.system(f"mkdir -p {name}")

In [ ]:
# @title 3. Upload a PDB file for MDPs (skip if your protein is an IDP)

# @markdown Please ensure that residues are numbered from 1 in the uploaded PDB file

if not isIDP:
    uploaded = files.upload()
    if len(list(uploaded.keys())) != 1:
        raise Exception("Please upload just one pdb file")
    for key in uploaded.keys():
        os.system(f"mv {key} {name}")
    path2pdb = f"{name}/{list(uploaded.keys())[0]}"
    print(f"{path2pdb} successfully uploaded.")
else:
    print("No need to upload a PDB file")

In [ ]:
# @title 4. Upload SAXS data
# @markdown SAXS data must be in a file containing 3 columns, which are q, I and sigma. Commented lines (#) are allowed. 

tmp = files.upload()
saxs_file = list(tmp.keys())[0]

#check data
try:
    np.loadtxt(saxs_file)
except:
    print("Unable to read file. Make sure the file only contains 3 columns (q,I,sigma) and #commented lines")
assert np.shape(np.loadtxt(saxs_file))[1] == 3, "Expected file with 3 columns (q,I,sigma)"

exp_saxs = np.loadtxt(saxs_file)
if exp_saxs[...,0][-1] <  1:
    print('q is in Å units. Converting to nm.')
    exp_saxs[...,0] = exp_saxs[...,0]*10
    np.savetxt(saxs_file, exp_saxs)

if (exp_saxs[...,0] >= 5).sum() > 0:
    print('Found {} q-values above 5 nm^(-1). SAXS calculations are not reliable in that region of the spectrum. Those datapoints will be remove'.format((exp_saxs[...,0] >= 5).sum()))
    exp_saxs = exp_saxs[(exp_saxs[...,0] < 5)]
    np.savetxt(saxs_file, exp_saxs)

shutil.move(saxs_file, 'saxs_input.dat')

In [ ]:
# @title 5. Determine simulation time (unit: ns)

#@markdown By checking the "break_CALVADOS" box, you will add random noise to the amino-acid stickiness parameters of the CALVADOS model ($\lambda$ values). The resulting simulations will not be trustworthy. This function has been added for teaching purposes, so that the effect of reweighting could be appreciated more when using a force field that does not reproduce experimental data accurately.
Break_CALVADOS = False #@param {type:"boolean"}
if Break_CALVADOS == True:
    print ("WARNING: Stickiness parameters have been randomly modified. This function is only for teaching purposes.")

#@markdown The default option “AUTO” will set the simulation time depending on sequence length. The longer the IDR, the larger the ensemble of conformations it can adopt. Moreover, the reconfiguration time of IDRs increases with increasing sequence length. Therefore, longer sequences will require more sampling. Typical simulation times range from ca. 5 min (a 71-ns-long simulation of an IDR of 70 residues), 7 min (71-ns-long simulation of an IDR of 140 residues), to 34 min for a 373-ns-long simulation of an IDR of 351 residues.
Simulation_time = "AUTO" #@param {type:"string"}
#@markdown <i>*Units: Simulation_time [ns]<i>
N_res = len(sequence)
if not isIDP:
    domain_len = 0
    for domain in domain_resNum:
        domain_len += len(domain)
    N_res -= domain_len

N_save = 3000 if N_res < 100 else int(np.ceil(3e-4 * N_res ** 2) * 1000)

if Simulation_time == "AUTO":
    N_steps = 210 * int(N_save)
    print(f'AUTO simulation length selected. Running for {N_steps*0.01/1000} ns, {N_steps} steps in total.')
else:
    N_steps = int(float(Simulation_time)*1000/0.01)
    print(f'Simulation length assigned. Running for {Simulation_time} ns, {N_steps} steps in total.')

nframes=N_steps // N_save
config_sim_data = dict(
temp=float(temperature), ionic=float(ionic_strength),
pH=float(pH), cutoff=cutoff,
L=L, wfreq=int(N_save), use_pdb=use_pdb,
path2pdb=path2pdb, use_hnetwork=use_hnetwork, domain_resNum=domain_resNum,
use_ssdomains=use_ssdomains, input_pae=input_pae,
k_restraint=k_restraint, protein_name=name,
gpu_id=0, N_res=int(N_res),
CoarseGrained="COM", isIDP=isIDP,
eps_factor=eps_factor,
CALVADOS_version=CALVADOS_version, seq=list(sequence), steps=N_steps,
discard_first_nframes=discard_first_nframes,
nframes=nframes, Break_CALVADOS=Break_CALVADOS)
yaml.dump(config_sim_data, open('config_sim.yaml', 'w'))

In [ ]:
#@title <b><font color='#A79AB2'>MD simulation toolbox</font></b>
import time
from openmm import app
from simtk import openmm, unit
import pandas as pd
import MDAnalysis as mda
import mdtraj as md
from fastprogress import master_bar, progress_bar

def geometry_from_pdb(protein_name, pdb, CoarseGrained="CA", ssdomains=None):
    """ positions in nm """
    u = mda.Universe(pdb)
    ini_CA = u.select_atoms("name CA")
    ini_CA.write(f"{protein_name}/ini_CA.pdb")
    if CoarseGrained == "CA":
        cas = u.select_atoms('name CA')  # in-place
        center_of_mass = cas.center_of_mass()
        cas.translate(-center_of_mass)
        pos = cas.positions / 10.  # nm, shape: (n, 3)
    elif CoarseGrained == "COM":  # COM
        pos = []
        resSeqinDomain = []
        for ssdomain in ssdomains:
            resSeqinDomain += ssdomain
        for i in range(len(u.residues)):  # i starts from 0
            if i+1 in resSeqinDomain:  # COM
                pos.append(u.residues[i].atoms.center_of_mass())
            else:  # CA
                pos.append(np.squeeze(u.residues[i].atoms.select_atoms("name CA").positions))
        center_of_mass = u.atoms.center_of_mass()
        pos = (np.array(pos) - center_of_mass) / 10.  # nm

    return pos, center_of_mass/10.  # nm

def genParamsDH(df,protein_name,prot,temp):
    kT = 8.3145*temp*1e-3
    fasta = prot.fasta.copy()
    r = df.copy()
    q = 1.0

    r.loc['H','q'] = q / ( 1 + 10**(prot.pH-6) )
    r.loc['X'] = r.loc[fasta[0]]
    r.loc['Z'] = r.loc[fasta[-1]]
    fasta[0] = 'X'
    fasta[-1] = 'Z'
    r.loc['X','q'] = r.loc[prot.fasta[0],'q'] + q
    r.loc['Z','q'] = r.loc[prot.fasta[-1],'q'] - q
    # Calculate the prefactor for the Yukawa potential
    fepsw = lambda T : 5321/T+233.76-0.9297*T+0.1417*1e-2*T*T-0.8292*1e-6*T**3
    epsw = fepsw(temp)
    lB = 1.6021766**2/(4*np.pi*8.854188*epsw)*6.022*1000/kT
    yukawa_eps = [r.loc[a].q*np.sqrt(lB*kT) for a in fasta]
    # Calculate the inverse of the Debye length
    yukawa_kappa = np.sqrt(8*np.pi*lB*prot.ionic*6.022/10)
    return yukawa_eps, yukawa_kappa

def genParamsLJ(df,protein_name,prot):
    fasta = prot.fasta.copy()
    r = df.copy()
    r.loc['X'] = r.loc[fasta[0]]
    r.loc['Z'] = r.loc[fasta[-1]]
    r.loc['X','MW'] += 2
    r.loc['Z','MW'] += 16
    fasta[0] = 'X'
    fasta[-1] = 'Z'
    types = list(np.unique(fasta))
    MWs = [r.loc[a,'MW'] for a in types]
    lj_eps = prot.eps_factor*4.184
    return lj_eps, fasta, types, MWs

def load_parameters(protein_name, CALVADOS_version, Break_CALVADOS, output=False):
    wget.download(f"https://github.com/KULL-Centre/ColabCALVADOS/raw/main/data/residues_CALVADOS{CALVADOS_version:d}.csv")
    os.system(f"mv residues_CALVADOS{CALVADOS_version}.csv {protein_name}/residues.csv")
    residues = pd.read_csv(f'{protein_name}/residues.csv').set_index('one', drop=False)
    if Break_CALVADOS:
        np.random.seed(seed=3100)
        residues['lambdas'] += np.random.normal(0,1,residues.index.size)
        residues['lambdas'] = (residues.lambdas-residues.lambdas.min())/(residues.lambdas.max()-residues.lambdas.min())
    residues.to_csv(f'{protein_name}/residues_pub.csv')
    if output:
        print("Lambda used:\n", residues.lambdas)
    return residues

def p2c(r, phi):
    """
    polar to cartesian
    """
    return (r * np.cos(phi), r * np.sin(phi))

def xy_spiral_array(n, delta=0, arc=.38, separation=.7):
    """
    create points on an Archimedes' spiral
    with `arc` giving the length of arc between two points
    and `separation` giving the distance between consecutive
    turnings
    """
    r = arc
    b = separation / (2 * np.pi)
    phi = float(r) / b
    coords = []
    for i in range(n):
        coords.append(list(p2c(r, phi))+[0])
        phi += float(arc) / r
        r = b * phi
    return np.array(coords)+delta

def build_topology(fasta,n_chains=1):
    # build CG topology
    top = md.Topology()
    for i_chain in range(n_chains):
        chain = top.add_chain()
        for resname in fasta:
            residue = top.add_residue(resname, chain)
            top.add_atom(resname, element=md.element.carbon, residue=residue)
        for i in range(chain.n_atoms-1):
            top.add_bond(chain.atom(i),chain.atom(i+1))
    return top

def build_box(Lx,Ly,Lz):
    # set box vectors
    a = unit.Quantity(np.zeros([3]), unit.nanometers)
    a[0] = Lx * unit.nanometers
    b = unit.Quantity(np.zeros([3]), unit.nanometers)
    b[1] = Ly * unit.nanometers
    c = unit.Quantity(np.zeros([3]), unit.nanometers)
    c[2] = Lz * unit.nanometers
    return a, b, c

def add_particles(system, residues, prot, n_chains=1):
    for i_chain in range(n_chains):
        system.addParticle((residues.loc[prot.fasta[0]].MW+2)*unit.amu)
        for a in prot.fasta[1:-1]:
            system.addParticle(residues.loc[a].MW*unit.amu)
        system.addParticle((residues.loc[prot.fasta[-1]].MW+16)*unit.amu)
    return system

def euclidean(a_matrix, b_matrix):
    # Using matrix operation to calculate Euclidean distance
    d1 = -2 * np.dot(a_matrix, b_matrix.T)
    d2 = np.sum(np.square(a_matrix), axis=1, keepdims=True)
    d3 = np.sum(np.square(b_matrix), axis=1)
    dist = np.sqrt(d1 + d2 + d3)
    return dist

def set_interactions(system, residues, prot, lj_eps, cutoff, yukawa_kappa, yukawa_eps, N, n_chains=1,
                     CoarseGrained="CA", dismatrix=None, isIDP=True, domain_resNum=None):
    hb = openmm.openmm.HarmonicBondForce()
    # interactions
    energy_expression = 'select(step(r-2^(1/6)*s),4*eps*l*((s/r)^12-(s/r)^6-shift),4*eps*((s/r)^12-(s/r)^6-l*shift)+eps*(1-l))'
    ah = openmm.openmm.CustomNonbondedForce(energy_expression + '; s=0.5*(s1+s2); l=0.5*(l1+l2); shift=(0.5*(s1+s2)/rc)^12-(0.5*(s1+s2)/rc)^6')
    ah.addGlobalParameter('eps', lj_eps * unit.kilojoules_per_mole)
    ah.addGlobalParameter('rc', float(cutoff) * unit.nanometer)
    ah.addPerParticleParameter('s')
    ah.addPerParticleParameter('l')
    print('rc', cutoff * unit.nanometer)

    yu = openmm.openmm.CustomNonbondedForce('q*(exp(-kappa*r)/r-shift); q=q1*q2')
    yu.addGlobalParameter('kappa', yukawa_kappa / unit.nanometer)
    yu.addGlobalParameter('shift', np.exp(-yukawa_kappa * 4.0) / 4.0 / unit.nanometer)
    yu.addPerParticleParameter('q')

    for j in range(n_chains):
        begin = j * N  # 0
        end = j * N + N  # n_residues
        for a, e in zip(prot.fasta, yukawa_eps):
            yu.addParticle([e * unit.nanometer * unit.kilojoules_per_mole])
            ah.addParticle([residues.loc[a].sigmas * unit.nanometer, residues.loc[a].lambdas * unit.dimensionless])

        for i in range(begin, end - 1):  # index starts from 0
            if CoarseGrained=="CA" or isIDP:
                hb.addBond(i, i + 1, 0.38 * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
            else:  # COM or COE
                ssdomains = domain_resNum
                resSeqinDomain = []
                for ssdomain in ssdomains:
                    resSeqinDomain += ssdomain
                if i+1 in resSeqinDomain or i+1+1 in resSeqinDomain:
                    hb.addBond(i, i + 1, dismatrix[i][i+1] * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
                else:
                    hb.addBond(i, i + 1, 0.38 * unit.nanometer, 8033.0 * unit.kilojoules_per_mole / (unit.nanometer ** 2))
            yu.addExclusion(i, i + 1)
            ah.addExclusion(i, i + 1)

    # All Forces in a single force group must use the same cutoff distance
    yu.setForceGroup(0)
    ah.setForceGroup(1)
    yu.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    ah.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    hb.setUsesPeriodicBoundaryConditions(True)
    yu.setCutoffDistance(4 * unit.nanometer)
    ah.setCutoffDistance(cutoff * unit.nanometer)
    return hb, yu, ah

def set_harmonic_network(N,dmap,pae_inv,yu,ah,ssdomains=None,cs_cutoff=0.9,k_restraint=700.):
    cs = openmm.openmm.HarmonicBondForce()
    for i in range(N-2):
        for j in range(i+2,N):
            if ssdomains != None:  # use fixed domain boundaries for network
                ss = False
                for ssdom in ssdomains:
                    if i+1 in ssdom and j+1 in ssdom:
                        ss = True
                if ss:  # both residues in structured domains
                    if dmap[i,j] < cs_cutoff:  # nm
                        cs.addBond(i, j, dmap[i, j] * unit.nanometer,
                                   k_restraint * unit.kilojoules_per_mole / (unit.nanometer ** 2))
                        yu.addExclusion(i, j)
                        ah.addExclusion(i, j)
            elif isinstance(pae_inv, np.ndarray):  # use alphafold PAE matrix for network
                k = k_restraint * pae_inv[i,j]**2
                if k > 0.0:
                    cs.addBond(i,j, dmap[i,j]*unit.nanometer,
                                k*unit.kilojoules_per_mole/(unit.nanometer**2))
                    yu.addExclusion(i, j)
                    ah.addExclusion(i, j)
            else:
                raise
    cs.setUsesPeriodicBoundaryConditions(True)
    return cs, yu, ah

def simulate(config):
    """ Simulate CALVADOS using OpenMM 

        * config is a dictionary """

    # parse config
    protein_name, temp, ionic = config['protein_name'], config['temp'], config['ionic']
    cutoff, steps, wfreq = config['cutoff'], config['steps'], config['wfreq']
    L = config['L']
    Break_CALVADOS = config["Break_CALVADOS"]
    domain_resNum = config['domain_resNum']
    eps_factor = config["eps_factor"]
    pH = config["pH"]
    isIDP = config['isIDP']
    CoarseGrained = config['CoarseGrained']
    seq = config['seq']
    replica = 0
    use_pdb, path2pdb = config['use_pdb'], config['path2pdb']
    CALVADOS_version = config['CALVADOS_version']
    center_of_mass = np.array([0, 0, 0])
    if use_pdb:
        use_hnetwork = config['use_hnetwork']
        if use_hnetwork:
            k_restraint = config['k_restraint']
            use_ssdomains = config['use_ssdomains']
            if use_ssdomains:
                ssdomains = domain_resNum
                pae = None
            else:
                raise Exception("please make sure 'use_ssdomains' is True")
    else:
        path2pdb = ''
        use_hnetwork = False
        pae = None
        ssdomains = None
    # load residue parameters
    residues = load_parameters(protein_name, CALVADOS_version, Break_CALVADOS, output=True)
    # build protein dataframe
    df = pd.DataFrame(columns=['pH', 'ionic', 'temp', 'eps_factor', 'fasta'], dtype=object)
    df.loc[protein_name] = dict(pH=pH, ionic=ionic, temp=temp, eps_factor=eps_factor, fasta=seq)
    prot = df.loc[protein_name]
    print("pH:", prot.pH)
    print(f"Ionic strength: {prot.ionic} M")
    print(f"Temperature: {prot.temp} K")
    # LJ and YU parameters
    lj_eps, fasta, types, MWs = genParamsLJ(residues, protein_name, prot)
    # print("lj_eps:", lj_eps * unit.kilojoules_per_mole)
    yukawa_eps, yukawa_kappa = genParamsDH(residues, protein_name, prot, temp)

    N = len(fasta)  # number of residues
    n_chains = 1
    Lz = L

    # get input geometry

    if use_pdb:
        print(f'Starting from pdb structure {path2pdb}')
        pos, center_of_mass = geometry_from_pdb(protein_name, path2pdb, CoarseGrained=CoarseGrained, ssdomains=domain_resNum)
    else:
        spiral = True
        if spiral:
            pos = xy_spiral_array(N)
        else:
            pos = [[L / 2, L / 2, L / 2 + (i - N / 2.) * .38] for i in range(N)]
        pos = np.array(pos)

    top = build_topology(fasta, n_chains=n_chains)
    md.Trajectory(pos + center_of_mass, top, 0, [L, L, Lz], [90, 90, 90]).save_pdb(f'{protein_name}/ini_beads.pdb', force_overwrite=True)
    pos = pos + np.array([L / 2, L / 2, L / 2])
    a = md.Trajectory(pos, top, 0, [L, L, Lz], [90, 90, 90])
    a.save_pdb(f'{protein_name}/top_production.pdb', force_overwrite=True)
    # build openmm system
    system = openmm.System()

    # box
    a, b, c = build_box(L, L, Lz)
    system.setDefaultPeriodicBoxVectors(a, b, c)

    # load topology into system
    pdb = app.pdbfile.PDBFile(f'{protein_name}/top_production.pdb')

    # print(pdb.topology)

    # particles and termini
    system = add_particles(system, residues, prot, n_chains=n_chains)
    dmap = euclidean(pos, pos)
    # interactions
    hb, yu, ah = set_interactions(system, residues, prot, lj_eps, cutoff, yukawa_kappa, yukawa_eps, N,
                                  n_chains=n_chains, CoarseGrained=CoarseGrained, dismatrix=dmap, isIDP=isIDP,
                                  domain_resNum=domain_resNum)
    system.addForce(hb)
    system.addForce(yu)
    system.addForce(ah)

    # print("use_hnetwork: ", use_hnetwork)
    # harmonic network (currently unavailable for slab sim)
    if use_hnetwork:
        cs, yu, ah = set_harmonic_network(N, dmap, pae, yu, ah, ssdomains=ssdomains, k_restraint=k_restraint)
        # print(f"k_restraint used: {k_restraint}")
        system.addForce(cs)

    open(f'{protein_name}/system.xml', 'w').write(openmm.XmlSerializer.serialize(system))
    # use langevin integrator
    integrator = openmm.openmm.LangevinMiddleIntegrator(temp * unit.kelvin, 0.01 / unit.picosecond, 0.01 * unit.picosecond)
    # print(integrator.getFriction(), integrator.getTemperature())

    try:
        platform = openmm.Platform.getPlatformByName('CUDA')
        simulation = app.simulation.Simulation(pdb.topology, system, integrator, platform, dict(CudaPrecision='mixed'))
        print("Using GPU")
    except openmm.OpenMMException:
        platform = openmm.Platform.getPlatformByName('CPU')
        simulation = app.simulation.Simulation(pdb.topology, system, integrator, platform)
        print("Using CPU")

    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    simulation.reporters.append(app.dcdreporter.DCDReporter(f'{protein_name}/production.dcd', wfreq, enforcePeriodicBox=False, append=False))

    simulation.reporters.append(
        app.statedatareporter.StateDataReporter(f'{protein_name}/statedata_production.log', int(wfreq),
                                                step=True, speed=True, elapsedTime=True, separator='\t', progress=True,
                                                remainingTime=True, totalSteps=steps))
    simulation.reporters.append(
        app.checkpointreporter.CheckpointReporter(file=f"{protein_name}/checkpoint_production.chk",
                                                  reportInterval=wfreq))


    starttime = time.time()  # begin timer
    # assert steps%wfreq==0
    print(f"Total frames: {steps//wfreq}; the first 10 frames will be discarded")
    for _ in progress_bar(list(range(steps//wfreq))):
        simulation.step(wfreq)
    endtime = time.time()  # end timer
    target_seconds = endtime - starttime  # total used time
    print(
        f"{protein_name} total simulations used time: {target_seconds // 3600}h {(target_seconds // 60) % 60}min {np.round(target_seconds % 60, 2)}s")

def fix_topology(dcd,pdb):
    """
    Changes atom names to CA; even if CG-strategy is COM, it is still CA for convenience
    """
    t = md.load_dcd(dcd,pdb)
    cgtop = md.Topology()
    cgchain = cgtop.add_chain()
    for atom in t.top.atoms:
        cgres = cgtop.add_residue(atom.name, cgchain)
        cgtop.add_atom('CA', element=md.element.carbon, residue=cgres)
    traj = md.Trajectory(t.xyz, cgtop, t.time, t.unitcell_lengths, t.unitcell_angles)
    traj = traj.superpose(traj, frame=0)
    return traj

def visualize_traj(protein_name):
    traj_dcd = fix_topology(f'{protein_name}/{protein_name}_cg.dcd', f'{protein_name}/{protein_name}_cg.pdb')
    traj_dcd = traj_dcd.superpose(traj_dcd[0])
    traj_dcd[0].save_pdb(f'{protein_name}/{protein_name}_first_cg.pdb')
    traj_dcd[-1].save_pdb(f'{protein_name}/{protein_name}_last_cg.pdb')
    traj_dcd.save_trr(f"{protein_name}/{protein_name}_cg.trr")
    traj_dcd.save_dcd(f"{protein_name}/{protein_name}_CA.dcd")

def centerDCD(config):
    protein_name = config["protein_name"]
    discard_first_nframes = config["discard_first_nframes"]
    CALVADOS_version = config["CALVADOS_version"]
    seq = config["seq"]
    Break_CALVADOS = config["Break_CALVADOS"]
    residues = load_parameters(protein_name, CALVADOS_version, Break_CALVADOS)
    top = md.Topology()
    chain = top.add_chain()
    for resname in seq:
        residue = top.add_residue(residues.loc[resname, 'three'], chain)
        top.add_atom(residues.loc[resname, 'three'], element=md.element.carbon, residue=residue)
    for i in range(len(seq) - 1):
        top.add_bond(top.atom(i), top.atom(i + 1))
    traj = md.load_dcd(f"{protein_name}/production.dcd", top=top)[discard_first_nframes:]
    traj = traj.image_molecules(inplace=False, anchor_molecules=[set(traj.top.chain(0).atoms)], make_whole=True)
    traj.center_coordinates()
    traj.xyz += traj.unitcell_lengths[0, 0] / 2
    # print(f'Number of frames: {traj.n_frames}')
    traj.save_dcd(f'{protein_name}/{protein_name}_cg.dcd')
    traj[0].save_pdb(f'{protein_name}/{protein_name}_cg.pdb')
    visualize_traj(protein_name)

In [ ]:
#@title <b><font color='#A79AB2'>Sequence analysis toolbox</font></b>
def calc_seq_prop(protein_name, seq, CALVADOS_version, Break_CALVADOS):
    """df: DataFrame to be populated with sequence properties
    r: DataFrame of aa-specific parameters"""
    url = 'https://github.com/KULL-Centre/_2023_Tesei_IDRome/blob/main'
    if os.path.exists('svr_model_nu.joblib') == False:
        wget.download(url+'/svr_models/svr_model_nu.joblib?raw=true')
    if os.path.exists('svr_model_SPR.joblib') == False:
        wget.download(url+'/svr_models/svr_model_SPR.joblib?raw=true')
    model_nu = load('svr_model_nu.joblib')
    model_spr = load('svr_model_SPR.joblib')
    r = load_parameters(protein_name, CALVADOS_version, Break_CALVADOS).set_index("three")

    seq = list(seq).copy()
    fasta_kappa = np.array(seq.copy())
    N = len(seq)
    r = residues.copy()

    # calculate properties that do not depend on charges
    fK = sum([seq.count(a) for a in ['K']])/N
    fR = sum([seq.count(a) for a in ['R']])/N
    fE = sum([seq.count(a) for a in ['E']])/N
    fD = sum([seq.count(a) for a in ['D']])/N
    faro = sum([seq.count(a) for a in ['W','Y','F']])/N
    mean_lambda = np.mean(r.loc[seq].lambdas)

    pairs = np.array(list(itertools.combinations(seq,2)))
    pairs_indices = np.array(list(itertools.combinations(range(N),2)))
    # calculate sequence separations
    ij_dist = np.diff(pairs_indices,axis=1).flatten().astype(float)
    # calculate lambda sums
    ll = r.lambdas.loc[pairs[:,0]].values+r.lambdas.loc[pairs[:,1]].values
    # calculate SHD
    beta = -1
    shd = np.sum(ll*np.power(np.abs(ij_dist),beta))/N

    # fix charges
    r.loc['X'] = r.loc[seq[0]]
    r.loc['X','q'] = r.loc[seq[0],'q'] + 1.
    seq[0] = 'X'
    if r.loc['X','q'] > 0:
        fasta_kappa[0] = 'K'
    else:
        fasta_kappa[0] = 'A'
    r.loc['Z'] = r.loc[seq[-1]]
    r.loc['Z','q'] = r.loc[seq[-1],'q'] - 1.
    seq[-1] = 'Z'
    if r.loc['Z','q'] < 0:
        fasta_kappa[-1] = 'D'
    else:
        fasta_kappa[-1] = 'A'
    if Hc < 0.5:
        r.loc['H', 'q'] = 0
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'A'
    elif Hc >= 0.5:
        r.loc['H', 'q'] = 1
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'K'

    # calculate properties that depend on charges
    pairs = np.array(list(itertools.combinations(seq,2)))
    # calculate charge products
    qq = r.q.loc[pairs[:,0]].values*r.q.loc[pairs[:,1]].values
    # calculate SCD
    scd = np.sum(qq*np.sqrt(ij_dist))/N
    SeqOb = SequenceParameters(''.join(fasta_kappa))
    kappa = SeqOb.get_kappa()
    fcr = r.q.loc[seq].abs().mean()
    ncpr = r.q.loc[seq].mean()

    nu_svr = model_nu.predict([[scd,shd,kappa,fcr,mean_lambda]])[0]
    spr_svr = model_spr.predict([[scd,shd,mean_lambda]])[0]

    return np.around([fK, fR, fE, fD, faro, mean_lambda, shd, scd,
                      kappa, fcr, ncpr, nu_svr, spr_svr],3)

In [ ]:
#@title <b><font color='#A79AB2'>Simulation analysis toolbox</font></b>

def calc_rg(t, protein_name, CALVADOS_version, Break_CALVADOS):
    df = load_parameters(protein_name, CALVADOS_version, Break_CALVADOS).set_index("three")
    masses = [df.loc[res.name,'MW'] for res in t.top.atoms]
    masses[0] += 2
    masses[-1] += 16
    # calculate the center of mass
    cm = np.sum(t.xyz*masses[np.newaxis,:,np.newaxis],axis=1)/masses.sum()
    # calculate residue-cm distances
    si = np.linalg.norm(t.xyz - cm[:,np.newaxis,:],axis=2)
    # calculate rg
    rgarray = np.sqrt(np.sum(si**2*masses,axis=1)/masses.sum())
    return rgarray
    
def calc_nu(traj,w=None):
    pairs = traj.top.select_pairs('all','all')
    d = md.compute_distances(traj,pairs)
    dmax = np.max(d)
    nres = traj.n_atoms
    ij = np.arange(2,nres,1)
    diff = [x[1]-x[0] for x in pairs]
    dij = np.empty(0)
    for i in ij:
        dij = np.append(dij,np.sqrt(np.average(np.mean(d[:,diff==i]**2,axis=1),weights=w,axis=0)))
    f = lambda x,R0,v : R0*np.power(x,v)
    popt, pcov = curve_fit(f,ij[ij>5],dij[ij>5],p0=[.4,.5])
    nu = popt[1]
    nu_err = pcov[1,1]**0.5
    R0 = popt[0]
    R0_err = pcov[0,0]**0.5
    return ij,dij,dmax,nu,nu_err,R0,R0_err

def calc_contact_map(t,df,seq,rc,temp,ionic):
    indices = t.top.select_pairs('all','all')
    mask = np.abs(indices[:,0]-indices[:,1])>1 #exclude bonded pairs
    indices = indices[mask]
    d = md.compute_distances(t,indices) #vector distances between pairs for each frame
    cmap = np.nansum((.5-.5*np.tanh((d-1.)/.3)),axis=0)
    df_cmap = pd.DataFrame(index=range(t.n_atoms),columns=range(t.n_atoms),dtype=float)
    for k,(i,j) in enumerate(pairs):
        df_cmap.loc[i,j] = cmap[k]
        df_cmap.loc[j,i] = cmap[k]
    return df_cmap

def calc_ree(t):
    return md.compute_distances( t, atom_pairs=np.array([[ 0,  len(list(t.top.atoms))-1]]) )[...,0]

def kde(a, w=None, phi_eff=None, min_=None, max_=None):
    if type(w) == 'NoneType':
        w = np.full(len(a), 1)
    if min_ == None:
        min_ = np.min(a)
    if max_ == None:
        max_ = np.max(a)
    x = np.linspace( min_, max_, num = 50 )
    d = scs.gaussian_kde( a, bw_method = "silverman", weights = w ).evaluate(x)
    u = DescrStatsW( a, weights = w )
    n_eff = phi_eff*a.size if phi_eff!=None else a.size
    return x,d/np.sum(d),u.mean,u.std/np.sqrt(n_eff)
    
def plot_dist(ax,x,p,av,color='k'):
    ax.plot(x,p,c=color)
    ax.axvline(av,color=color)
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(0,np.max(p)*1.1)

def plot_rew_dist(ax,x,p,av):
    ax.plot(x,p,c='tab:red')
    ax.axvline(av,0,100, color='tab:red')
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(0,np.max(p)*1.1)

def error_ratio(v1,v2,e1,e2):
    ratio = v1/v2
    return ratio*np.sqrt((e1/v1)**2+(e2/v2)**2)

In [ ]:
#@title <b><font color='#FA003F'>BME Toolbox</font></b>
def iBME(calc_file,exp_file,THETAS=np.array([1,10,20,50,75,100,200,400,750,1000,5000,10000])):
    W = []
    STATS = []

    out = display(progress(0, THETAS.size), display_id=True)

    for i,t in enumerate(THETAS):
        print(f'Reweighting with theta={t:d}')
        rew = BME.Reweight('ibme_t{}'.format(t))
        rew.load(exp_file,calc_file)
        rew.ibme(theta=t, iterations=25, ftol=0.001)

        W.append(rew.get_ibme_weights()[-1])
        STATS.append(rew.get_ibme_stats()[-1])
        out.update(progress(i, THETAS.size))
        #print('chi2={:.2f}, phi_eff={:.2f}'.format(STATS[-1][1],STATS[-1][2]))

    return THETAS, np.array(STATS), np.array(W)

def theta_loc(thetas, stats):
    kneedle = KneeLocator(stats[...,2], stats[...,1], S=1, curve="convex", direction="increasing")
    choice = np.array(thetas)[stats[...,2]==kneedle.knee][0]
    return choice

In [ ]:
# @title 5. Run MD simulation
config = yaml.safe_load(open("config_sim.yaml", 'r'))
simulate(config)
centerDCD(config)

In [ ]:
# @title 6. Analysis

t = md.load_dcd(f"{protein_name}/{protein_name}_cg.dcd", f"{protein_name}/{protein_name}_cg.pdb")
config = yaml.safe_load(open("config_sim.yaml", 'r'))

rg_array = calc_rg(t, config["protein_name"], config["CALVADOS_version"], config["Break_CALVADOS"])
ree_array = calc_ree(t)
ij,dij,dmax,nu,nu_err,R0,R0_err = calc_nu(t)
df_cmap = calc_contact_map(t)

rg, rg_err, rg_blocksize = autoblock(rg_array)
x_rg, p_rg, _, _ = kde(rg_array)
x_D, p_D, _, _ = kde(D_array)
x_S, p_S, _, _ = kde(S_array)

ree_array = Ree(traj)
ree, ree_err, ree_blocksize = autoblock(ree_array)
x_ree, p_ree, _, _ = kde(ree_array)

# Plot results
mpl.rcParams.update({'font.size': 10})
fig, axs = plt.subplots(2, 3, figsize=(7,3.5), facecolor='w', dpi=300, layout='tight')
axs = axs.flatten()

axs[0].plot(x_rg, p_rg)
top = p_rg.max()+0.1*p_rg.max()
axs[0].axvline(rg)

axs[0].set_xlabel(r'$R_g$ (nm)')
axs[0].set_ylabel(r'$p(R_g)$')
axs[0].set_ylim(0,top)
axs[0].fill_between([rg-rg_err,rg+rg_err],0,top,alpha=0.3)

axs[1].plot(x_D, p_D)
top = p_D.max()+0.1*p_D.max()
axs[1].axvline(D)
axs[1].set_xlabel(r'$\Delta$')
axs[1].set_ylabel(r'$p(\Delta)$')
axs[1].set_ylim(0,top)
axs[1].fill_between([D-D_err,D+D_err],0,top,alpha=0.3)

axs[3].plot(x_ree, p_ree)
top = p_ree.max()+0.1*p_ree.max()
axs[3].vlines(ree,0,top)
axs[3].set_xlabel(r'$R_{ee}$ (nm)')
axs[3].set_ylabel(r'$p(R_{ee})$')
axs[3].set_ylim(0,top)

im = axs[4].imshow(df_cmap*1e3,extent=[1, df_cmap.shape[0], 1, df_cmap.shape[0]],origin='lower',
                   aspect='equal',vmin=0,vmax=1,cmap=plt.cm.Blues_r)
cb = plt.colorbar(im, ax=axs[4], label='Contacts',fraction=0.05, pad=0.04)
cb.set_ticks([0,-3,-6,-9])
axs[4].set_xlabel('Residue #')
axs[4].set_ylabel('Residue #')

if config["isIDP"]:

    axs[5].plot(ij,dij)
    dij_fit = R0*np.power(ij,nu)
    axs[5].plot(ij, dij_fit,c='0.3',ls='dashed',label='Fit')
    axs[5].set_xlabel('$|i-j|$')
    axs[5].set_ylabel(r'$\sqrt{\langle R_{ij}^2 \rangle}$ (nm)')
    axs[5].text(0.05, 0.9, r'$\nu$={:.2f}'.format(nu), horizontalalignment='left',
                verticalalignment='center', transform=axs[4].transAxes, fontsize=10)
    axs[5].legend(loc='lower right')

plt.savefig(f'{config["protein_name"]}/conformational_properties.pdf', 
            dpi=300, facecolor='w', edgecolor='w', orientation='portrait',
            bbox_inches='tight')
plt.show()

if config["isIDP"]:
    df_means = pd.DataFrame(data=np.c_[[rg,rg_err],[ree,ree_err],[nu,nu_err]],
                        columns=['<Rg> (nm)','<Ree> (nm)','nu'],
                        index=['Value','Error'])
else:
    df_means = pd.DataFrame(data=np.c_[[rg,rg_err],[ree,ree_err]],
                        columns=['<Rg> (nm)','<Ree> (nm)'],
                        index=['Value','Error'])

df_means.to_csv(f'{config["protein_name"]}/conf_properties.csv')
df_cmap.to_csv(f'{config["protein_name"]}/contact_map.csv')

df_means

In [ ]:
#@title <b><font color='#F26419'>8. Sequence analysis</font></b>
#@markdown This cell calculates the following sequence descriptors: fractions of K, R, D, E, and aromatic residues; average stickiness, $\langle \lambda \rangle$; sequence hydropathy decorator, SHD; sequence charge decorator, SCD; charge segregation parameter $\kappa$; fraction of charged residues, FCR; and net charge per residue, NCPR. For IDRs, $\langle \lambda \rangle$, SHD, SCD, $\kappa$, and FCR will be used to estimate the apparent Flory scaling exponent, $\nu_\text{SVR}$, using a machine learning model trained on simulations of tens of thousands of sequences of human IDRs (see https://doi.org/10.1038/s41586-023-07004-5).
df_seq = pd.DataFrame(columns=['fK','fR','fE','fD','fARO','mean_lambda','SHD','SCD','kappa',
                           'FCR','NCPR','nu_SVR','S_conf,SVR/N (kB)'])

df_seq.loc[config['protein_name']] = calc_seq_prop(config["protein_name"], 
                    config["seq"], config["CALVADOS_version"], config["Break_CALVADOS"])

if not config["isIDP"]:
    df_seq.drop(['nu_SVR','S_conf,SVR/N (kB)'],axis=1)
    
df_seq.to_csv(f'{config["protein_name"]}/sequence_properties.csv')
df_seq

In [ ]:
# @title 9. Visualize trajectory
import py3Dmol
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["resid"] = int(int(line[22:26]))
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip()
    def __str__(self):
        line = list(" " * 80)

        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].ljust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].ljust(3)
        line[22:26] = str(self["resid"]).ljust(4)
        line[30:38] = str(self["x"]).rjust(8)
        line[38:46] = str(self["y"]).rjust(8)
        line[46:54] = str(self["z"]).rjust(8)
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"

class Molecule(list):
    def __init__(self, file):
        for line in file:
            if "ATOM" in line or "HETATM" in line:
                self.append(Atom(line))

    def __str__(self):
        outstr = ""
        for at in self:
            outstr += str(at)

        return outstr

if not config["isIDP"]:
    all_domain_residues = []
    for temp in domain_resNum:
        all_domain_residues += temp
    template = Molecule(open(f'{config["protein_name"]}/{config["protein_name"]}_cg.pdb', 'r'))
    for at in template:
        if int(at["idx"]) in all_domain_residues:
            at["pymol"] = {"sphere": {'color': "red"}}

md.load_dcd(f'{config["protein_name"]}/{config["protein_name"]}_cg.dcd', f'{config["protein_name"]}/{config["protein_name"]}_cg.pdb').save_pdb(f'{config["protein_name"]}/{config["protein_name"]}_view.pdb')
traj = md.load_pdb(f'{config["protein_name"]}/{config["protein_name"]}_view.pdb')

if not config["isIDP"]:
    domain_resNum = config["domain_resNum"]
    traj = traj.superpose(traj[0], atom_indices=np.array(domain_resNum[0]))
    traj.save_pdb(f'{config["protein_name"]}/{config["protein_name"]}_view.pdb')
with open(f'{config["protein_name"]}/{config["protein_name"]}_view.pdb') as ifile:
    view = py3Dmol.view(width=400, height=300)
    view.addModelsAsFrames("".join([x for x in ifile]))
    if not config["isIDP"]:
        for i, at in enumerate(template):
            default = {"sphere": {'color': 'cyan'}}
            view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
    else:
        view.setStyle({'model': -1}, {"sphere": {'color': 'cyan'}})
    view.zoomTo()
    view.animate({'loop': "forward"})
    view.show()

In [ ]:
# @title Install cg2all for all-atom reconstruction (5 mins)
%%bash
pip install dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html &> /dev/null
pip install -q git+http://github.com/huhlim/cg2all@cuda-12 &> /dev/null
pip install -q py3Dmol gdown mrcfile &> /dev/null

In [ ]:
# @title Install Pepsi-SAXS and download BLOCKING, BME, and BIFT
wget https://files.inria.fr/NanoDFiles/Website/Software/Pepsi-SAXS/Linux/3.0/Pepsi-SAXS-Linux.zip &> /dev/null
unzip Pepsi-SAXS-Linux.zip &> /dev/null
rm Pepsi-SAXS-Linux.zip

wget https://raw.githubusercontent.com/fpesceKU/BLOCKING/main/block_tools.py &> /dev/null
wget https://raw.githubusercontent.com/fpesceKU/BLOCKING/main/main.py &> /dev/null

wget https://raw.githubusercontent.com/KULL-Centre/BME/main/BME_tools.py &> /dev/null
wget https://raw.githubusercontent.com/KULL-Centre/BME/main/BME.py &> /dev/null

wget https://raw.githubusercontent.com/ehb54/GenApp-BayesApp/main/bin/source/bift.f &> /dev/null
gfortran bift.f -march=native -O2 -o bift

In [ ]:
#@title Reconstruct all-atom trajectory
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import yaml
config = yaml.safe_load(open("config_sim.yaml", 'r'))
protein_name = config["protein_name"]
isIDP = config["isIDP"]
cg_model = "CalphaBasedModel" if isIDP else "ResidueBasedModel"
print(f"using {cg_model}")
!convert_cg2all -p {protein_name}/{protein_name}_first_cg.pdb -d {protein_name}/{protein_name}_cg.dcd -o {protein_name}/{protein_name}_AA.dcd -opdb {protein_name}/{protein_name}_topAA.pdb --cg {cg_model}

In [ ]:
#@title <b><font color='#058ED9'>3 - Ensemble reweighting against experimental data</b></font>
#@markdown The Bayesian/Maximum-entropy approach is used to reweight the MD simulations so that it better matches the SAXS data. This is done by minimizing the functional, $L(w_1 ... w_n) = \frac{m}{2}\chi^2(w_1 ... w_n)-\theta\; S_{rel}(w_1 ... w_n)$, where $(w_1 ... w_n)$ are the statistical weights associated with each frame of the simulation, $\chi^2$ quantifies the agreement between simulation and SAXS, $S_{rel}$ quantifies how much the new weights are different from the initial ones. $\theta$ is a free parameter that must be tuned to strike a balance between obtaining a good agreement with the experimental data (low $\chi^2$) and retaining as much information as possible from the starting simulation (high $S_{rel}$).
#correct experimental errors with BIFT
f = open('inputfile.dat','w')
f.write("saxs_input.dat\n\n\n\n{}\n\n\n\n\n\n\n\n\n\n\n\n".format(dmax))
f.close()
subprocess.run('./bift < inputfile.dat'.split())
np.savetxt('{:s}_bift.dat'.format(config["protein_name"]), np.loadtxt('saxs_input.dat'), header=' DATA=SAXS')
print('Experimental errors on SAXS intensities have been corrected with BIFT')
print('Factor used for rescaling errors is: {}'.format(np.loadtxt('scale_factor.dat')[0,1]))
print('SAXS data with corrected errors is in {:s}_bift.dat\n'.format(config["protein_name"]))

#@markdown 1. We calculate SAXS curves for each trajectory frame using Pepsi-SAXS.
#SAXS
print('Calculating SAXS from all-atom trajectory...')
t_aa = md.load_dcd(f'{config["protein_name"]}/traj_AA.dcd', 
                   top=f'{config["protein_name"]}/top_AA.pdb')
for i,f in enumerate(t_aa):
    f.save_pdb('frame.pdb')
    pepsi_comm = './Pepsi-SAXS frame.pdb {:s}_bift.dat -o saxs.dat -cst --cstFactor 0 --I0 1.0 --dro 1.0 --r0_min_factor 1.025 --r0_max_factor 1.025 --r0_N 1'.format(NAME)
    subprocess.run(pepsi_comm.split())
    if i == 0:
        calc_saxs = np.loadtxt('saxs.dat')[...,3]
    else:
        calc_saxs = np.vstack((calc_saxs,np.loadtxt('saxs.dat')[...,3]))
col0 = np.arange(0,len(calc_saxs)).reshape(len(calc_saxs),1)
calc_saxs = np.hstack((col0,calc_saxs))
np.savetxt('calc_saxs.dat', calc_saxs)

#@markdown 2. We execute BME reweighing using different $\theta$ values. For each $\theta$ value, we calculate $\chi^2$ and $\phi_{eff}=\exp(S_{rel})$.
def f(theta):
    mpl.rcParams.update({'font.size': 10})
    fig = plt.figure(layout='constrained',dpi=300,figsize=(7,3))
    plt.scatter(stats[...,2],stats[...,1], c='k')
    plt.scatter(stats[...,2][theta],stats[...,1][theta], c='tab:red',label=r'Chosen $\theta$')
    plt.xlabel(r'$\phi_{eff}$',fontsize=10)
    plt.ylabel(r'$\chi^2_r$',fontsize=10)
    plt.legend()
    plt.show()
    return theta

thetas, stats, weights = iBME('calc_saxs.dat', '{:s}_bift.dat'.format(NAME))

In [ ]:
#@title <b><font color='#058ED9'>3.1 - Setting $\theta$</b></font>
#@markdown This cell plots $\chi^2$ vs $\phi_{eff}=\exp(S_{rel})$ for the different $\theta$ values scanned in the previous cell. The “THETA_LOCATOR” option identifies the optimal $\theta$ value located at the elbow of the curve. Switching the “THETA_LOCATOR” option from “AUTO” to “INTERACTIVE”, you can use a drop-down menu to select a specific $\theta$ value to use. After running cell 3.2 below, try selecting different $\theta$ values, both high and low, and run cell 3.2 again. How do the structural observables and the fit to SAXS change in response to changes in $\theta$?

THETA_LOCATOR = "AUTO" #@param ["AUTO", "INTERACTIVE"]
if THETA_LOCATOR == "AUTO":
    choice = theta_loc(thetas, stats)
    mpl.rcParams.update({'font.size': 10})
    fig = plt.figure(layout='constrained',dpi=300,figsize=(7,3))
    plt.scatter(stats[...,2],stats[...,1], c='k')
    ndx = np.where(thetas==choice)[0][0]
    plt.scatter(stats[...,2][ndx],stats[...,1][ndx], c='tab:red',label=r'Chosen $\theta$')
    plt.xlabel(r'$\phi_{eff}$',fontsize=10)
    plt.ylabel(r'$\chi^2_r$',fontsize=10)
    plt.legend()
    plt.show()
elif THETA_LOCATOR == "INTERACTIVE":
    interactive_plot = interactive(f, theta=list(zip(thetas,range(thetas.size))))
    display(interactive_plot)

In [ ]:
#@title <b><font color='#058ED9'>3.2 Analyze reweighted ensemble</b></font>
#@markdown This cell shows comparisons between SAXS curves and conformational properties from experiments (grey) and from the simulation trajectory before (blue) and after BME reweighting (red). As an exercise, go back to cell 1.1 and check the "Break_CALVADOS" box. That will add random noise to the amino-acid stickiness parameters of the CALVADOS model ($\lambda$ values). The resulting force field will likely not reproduce the experimental data accurately and the effect of reweighting will be more evident.
if THETA_LOCATOR == "INTERACTIVE":
    ndx = interactive_plot.result
    choice = thetas[ndx]

print('Reweighting using theta={}'.format(choice))

q, I_exp, err = np.loadtxt('{}_bift.dat'.format(NAME), unpack=True)
I_prior = np.average(np.loadtxt('calc_saxs.dat')[...,1:],axis=0)
I_post = np.average(np.loadtxt(list(filter(lambda x: x.startswith('ibme_t{}_'.format(choice)) and x.endswith('.calc.dat'), os.listdir('.')))[0])[...,1:], axis=0, weights=weights[ndx])
wlr = 1/(err**2)
model = LinearRegression()
model.fit(I_prior.reshape(-1,1),I_exp,wlr)
a = model.coef_[0]
b = model.intercept_
I_prior = a*I_prior+b

mpl.rcParams.update({'font.size': 10})
fig, axs = plt.subplots(3, 2, figsize=(7,6), facecolor='w', dpi=300, layout='constrained')
axs = axs.flatten()

axs[0].errorbar(q,I_exp,err,c='tab:gray',alpha=.5)
axs[0].plot(q,I_prior, zorder=500)
axs[0].plot(q,I_post, color='tab:red', zorder=1000)
axs[0].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[0].set_ylabel('Intensity')
axs[0].set_ylim(0,I_post.max()*1.1)
axs[0].set_xlim(0,4)

axs[1].errorbar(q,I_exp,err,lw=1,c='tab:gray',alpha=.5)
axs[1].plot(q,I_prior, zorder=500)
axs[1].plot(q,I_post, color='tab:red', zorder=1000)
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[1].set_ylabel('Intensity')
axs[1].set_ylim(0,I_prior.max()*1.1)

kratky_exp = (q**2)*I_exp
kratky_err = (q**2)*err
axs[2].errorbar(q,kratky_exp,kratky_err,c='tab:gray',alpha=.5)
axs[2].plot(q,(q**2)*I_prior, zorder=500)
axs[2].plot(q,(q**2)*I_post, color='tab:red', zorder=1000)
axs[2].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[2].set_ylabel(r'$q^2I$')
axs[2].set_ylim(0,((q**2)*I_post).max()*1.1)
axs[2].set_xlim(0,4)

axs[3].plot(q, (I_exp-I_prior)/err, color='tab:blue', lw=1)
axs[3].plot(q, (I_exp-I_post)/err, color='tab:red', lw=1)
axs[3].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[3].set_ylabel(r'$(I^\mathrm{EXP}-I^\mathrm{CALC})/\sigma$')

x_rg_rew, p_rg_rew, rg_av_rew, rg_se_rew = kde(rg_array, w=weights[ndx], phi_eff=stats[...,2][ndx], min_=np.min(rg_array), max_=np.max(rg_array))
x_ree_rew, p_ree_rew, ree_av_rew, ree_se_rew = kde(ree_array, w=weights[ndx], phi_eff=stats[...,2][ndx], min_=np.min(ree_array), max_=np.max(ree_array))
ij, dij_rew, dmax_rew, nu_rew, nu_err_rew, _, _ = calc_nu(traj, w=weights[ndx])

plot_dist(axs[4], x_rg, p_rg, rg,color='tab:blue')
plot_rew_dist(axs[4], x_rg_rew, p_rg_rew, rg_av_rew)

axs[4].set_xlabel(r'$R_g$ (nm)')
axs[4].set_ylabel(r'$p(R_g)$')
axs[4].annotate(xy=(0.95,0.85), xycoords='axes fraction', text=r'$\langle R_g \rangle={:.2f}$ nm'.format(rg_av_rew), color='tab:red', fontsize=10, horizontalalignment='right')

axs[5].plot(ij,dij,c='tab:blue')
axs[5].plot(ij,dij_rew,nu_rew,c='tab:red')
axs[5].set_xlabel('$|i-j|$')
axs[5].set_ylabel(r'$\langle R_{ij} \rangle$ (nm)')
axs[5].annotate(xy=(0.05,0.85), xycoords='axes fraction', text=r'$\nu$={:.2f}'.format(nu_rew), color='tab:red', fontsize=10)

try:
    os.mkdir(f'{config["protein_name"]}/REWEIGHTED')
except:
    pass

plt.savefig(f'{config["protein_name"]}/REWEIGHTED/SAXS_reweighting.pdf', dpi=300, facecolor='w', edgecolor='w', orientation='portrait',
            bbox_inches='tight')
plt.show()

In [ ]:
# @title 8. Download results

# @markdown In this zip file:

# @markdown `${protein_name}_cg.dcd`: the coarse-grained trajectory file;

# @markdown `${protein_name}_cg.pdb`: the coarse-grained topology file;

# @markdown After reconstruction (Optional)

# @markdown `${protein_name}_AA.dcd`: the reconstructed trajectory file;

# @markdown `${protein_name}_topAA.pdb`: the all-atom topology file;
import subprocess
zipper = f'zip -r {name}.zip {name}'
subprocess.run(zipper.split())
files.download(f'{name}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>